<a href="https://colab.research.google.com/github/alexaK88/Q_jpeg_pennylane/blob/main/full_flow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pennylane
!pip install pennylane pennylane-lightning[gpu]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 935.6/935.6 kB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 913.3/913.3 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.2/581.2 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.5/366.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 

### Import Libraries

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pennylane as qml
from pennylane.templates import QFT
from sklearn.svm import SVC
from sklearn.datasets import fetch_openml, load_digits
from sklearn.preprocessing import MinMaxScaler, normalize
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from pennylane import numpy as pnp
from skimage.transform import resize
from keras.datasets import mnist

### Step 1:  Dataset Preparation


First, we load the MNIST dataset from openML.
- X is the pixel data
- y is the labels
- converting everything to `uint8` here to ensure all values are integers in [0, 255]

In [3]:
# loading mnist from openML
mnist = fetch_openml('mnist_784', version=1, cache=True)
X = mnist['data'].astype(np.uint8) # better to convert for binerization
y = mnist['target'].astype(np.uint8)
y = y.to_numpy()

Next, we focus on 2 classes, i.e. binary classification.
Here, I've been experimenting with different classes, and I stopped on 4 vs 9, cause they have more subtle difference in pixels, they are similar looking.

In [4]:
# focus on binary classification
mask = (y == 4) | (y == 9)
X, y = X[mask], y[mask]
X.shape

(13782, 784)

- I take only the first `n_samples`.
- I convert X to a NumPy array, and shuffling the data randomly

In [5]:
n_samples = 100 # restricting to 6000 samples for now

X = X.values if hasattr(X, "values") else X # safer conversion
perm = np.random.permutation(len(X))
X, y = X[perm], y[perm]

X = X[:n_samples]
y = y[:n_samples]

Now, I normalise pixel intensities.
- [0, 255] -> [0, 1]
- reshaping images back to 2D for resizing, i.e to 28x28 array with float values between 0 and 1.

In [6]:
X = X / 255.0
X = X.reshape(-1, 28, 28)

print(X.shape)
print("Pixel range:", X.min(), X.max())

(100, 28, 28)
Pixel range: 0.0 1.0


And now I reduce images to 8x8 + flattening to (, 64)
- resize -> flatten -> normalize

In [7]:
# convert each 28x28 binarised image to 8x8, then flatten to length 64
def to_8x8_vector(img_row):
    img_8x8 = resize(
        img_row,
        (8, 8),
        anti_aliasing=False,
        preserve_range=True,
        order=1 # controlling interpolation
    )
    img_8x8 = img_8x8.flatten()
    s = np.sum(img_8x8)

    if s > 0:
        img_8x8 = np.sqrt(img_8x8 / s)
    else:
        img_8x8 = np.zeros_like(img_8x8)
        img_8x8[0] = 1.0
      # should be shape (64,)
    return img_8x8

# apply transformation to all images
X_8x8 = np.array([to_8x8_vector(x) for x in X], dtype=float)
X_8x8.shape

(100, 64)

In [8]:
# sanity check, make sure no NaNs exist and all vectors are normalised, i.e. norm is around 1
print("Any NaNs?", np.isnan(X_8x8).any())
print("Norm check:", np.min(np.linalg.norm(X_8x8, axis=1)), np.max(np.linalg.norm(X_8x8, axis=1)))

Any NaNs? False
Norm check: 0.9999999999999999 1.0000000000000002


I'm gonna do the splitting here, and carry both representations consistently.
- qek inputs: (64,) flattened and normalized vectors, for quantum kernel embedding
- qjpeg: 28x28 images

In [9]:
idx = np.arange(n_samples)

idx_train, idx_test, y_train, y_test = train_test_split(
    idx, y, test_size=0.2, random_state=42, stratify=y
)

# QEK inputs (8x8 -> 64 -> normed)
X_train_qek = X_8x8[idx_train]
X_test_qek  = X_8x8[idx_test]

# QJPEG inputs (28x28 binary images)
X_train_img = X[idx_train]
X_test_img  = X[idx_test]

print("QEK train/test:", X_train_qek.shape, X_test_qek.shape)
print("IMG train/test:", X_train_img.shape, X_test_img.shape)
print("Labels train/test:", y_train.shape, y_test.shape)

QEK train/test: (80, 64) (20, 64)
IMG train/test: (80, 28, 28) (20, 28, 28)
Labels train/test: (80,) (20,)


Data preparation is done.

### Step 2: Quantum Embedding & Kernel Training

In [10]:
device = "lightning.qubit"

Defining number of qubits and device.

In [11]:
n_qubits = 6
wires = range(n_qubits)

dev = qml.device(device, wires=wires, shots=None)

Defining the circuit - feature map

In [12]:
def feature_map(x, theta):
    # Amplitude encoding (always normalized)
    qml.AmplitudeEmbedding(x, wires=wires, normalize=False)

    # Variational re-uploading layers
    for l in range(theta.shape[0]):
        for i in range(n_qubits):
            qml.RX(theta[l, i, 0], wires=i)
            qml.RY(theta[l, i, 1], wires=i)
            qml.RZ(theta[l, i, 2], wires=i)

        # Entangling layer (ring or chain)
        for i in range(n_qubits - 1):
            qml.CNOT(wires=[i, i + 1])
        qml.CNOT(wires=[n_qubits - 1, 0])

@qml.qnode(dev, interface="autograd")
def kernel_qnode(x1, x2, theta):
    feature_map(x1, theta)
    qml.adjoint(feature_map)(x2, theta)
    return qml.expval(qml.Projector([0]*n_qubits, wires=wires))


In [13]:
n_qubits = 6
n_layers = 4          # number of trainable layers
layers = n_layers     # just to match variable names
batch_size = 8       # bigger batch for stability
n_steps = 50          # training steps
stepsize = 0.02       # smaller learning rate
eps_trace = 1e-8      # small epsilon to prevent division by zero

In [14]:
def kernel_matrix(X1, X2, theta):
    return qml.math.stack([
        qml.math.stack([
            kernel_qnode(x1, x2, theta)
            for x2 in X2
        ])
        for x1 in X1
    ])

def kernel_alignment_loss_batch(theta, X, y, batch_size):
    idx = np.random.choice(len(X), batch_size, replace=False)
    Xb = X[idx]
    yb = y[idx]

    # Kernel
    K = kernel_matrix(Xb, Xb, theta)

    # Labels: {0,1} → {−1,+1}
    y_pm = 2 * yb - 1
    yy = qml.math.outer(y_pm, y_pm)

    # Center kernel
    n = batch_size
    H = qml.math.eye(n) - qml.math.ones((n, n)) / n
    Kc = H @ K @ H

    # Flatten
    Kf = qml.math.reshape(Kc, (-1,))
    yyf = qml.math.reshape(yy, (-1,))

    # Kernel Target Alignment
    numerator = qml.math.dot(Kf, yyf)
    denominator = qml.math.sqrt(
        qml.math.dot(Kf, Kf) * qml.math.dot(yyf, yyf)
    )

    return -numerator / denominator

### Step 3: QJPEG Compression

In [15]:
def vectorization(img, Cr, Cc, renorm=False):
    "Vectorize the image into amplitude-encoding patches suitable for quantum circuits"
    # splitting the original image (Mr, Mc) into S equal-size patches of shape (Cr, Cc)
    Mr, Mc = img.shape
    assert Mr % Cr == 0 and Mc % Cc == 0
    patches = (img.reshape(Mc//Cr, Cr, -1, Cc).swapaxes(1, 2).reshape(-1, Cr, Cc))
    # 64 patches, (64, 64, 64) shape; S=64

    # vectorize each patch and collect all in a (N, Cr*Cc) array
    vect_patches = np.reshape(patches,  (patches.shape[0], Cr*Cc)) # (64, 4096)

    # normalize each (Cr*Cc) vector to the intensity of the corresponding (Cr, Cc) patch
    states = np.zeros((patches.shape[0], Cr*Cc)) # (64, 4096)
    norm = np.zeros(patches.shape[0])

    for idx in range(patches.shape[0]): # for each patch
        # compute the sum of pixels intensities
        norm[idx] = vect_patches[idx].sum()
        if norm[idx] == 0:
            # empty patch -> encode |0...0>
            states[idx, 0] = 1.0
            norm[idx] = 1.0
            continue

        # normalize the patch vector so that its entries sum is 1
        tmp = vect_patches[idx] / norm[idx]
        # take the element-wise square root of the normalized vector
        states[idx] = np.sqrt(tmp)
    if renorm == False:
        norm = np.ones(patches.shape[0])
    print(states[:10])

    return states, norm # amplitudes, pixel intensities' sums

In [16]:
def qft_swaps(wires):
    n = len(wires)
    # apply QFT to all qubits
    qml.QFT(wires=wires)
    # add swaps to reverse qubit order!
    for i in range(n // 2):
        qml.SWAP(wires=[wires[i], wires[n - i - 1]])


def iqft_swaps(wires):
    n = len(wires)
    # swaps again - BEFORE iqft
    for i in reversed(range(n // 2)):
        qml.SWAP(wires=[wires[i], wires[n-i-1]])
    qml.adjoint(QFT)(wires=wires)

In [17]:
def circuit_builder(states, n0, n2, shots):
    ntilde = (n0 - n2) // 2
    n1 = n0 - ntilde

    qnodes = []

    # define device with n0 qubits
    dev = qml.device(device, wires=n0, shots=shots)

    for idx in range(states.shape[0]):
        # qnode to capture current input state
        @qml.qnode(dev)
        def circuit():
            # print("State norm:", np.linalg.norm(states[idx]))
            # initializing the state (using AmplitudeEmbedding here, but I'm wondering if something else could work faster)
            qml.AmplitudeEmbedding(states[idx], wires=range(n0), normalize=True)

            # Hadamard on all n0 qubits
            for w in range(n0):
                qml.Hadamard(wires=w)

            # apply QFT on all qubits
            qft_swaps(wires=range(n0))

            # apply IQFT on first n1 qubits
            iqft_swaps(wires=range(n1))

            # setting boundaries - Rule 2
            discard_start = n0 // 2 - ntilde
            discard_end = n0 // 2 - 1
            discarded_qubits = set(range(discard_start, discard_end + 1))

            # keep exactly n2 qubits for output
            measured_qubits = list(range(n2))


            # Hadamard on remaining qubits
            for q in measured_qubits:
                qml.Hadamard(wires=q)

            # print(f'Measured qubits: {measured_qubits}')

            return qml.probs(wires=measured_qubits)
        qnodes.append(circuit)

    return qnodes



In [18]:
def reconstruction(qnodes, n2, norm):
    out_freq = np.zeros((len(qnodes), 2**n2))
    for idx, qnode in enumerate(qnodes):
        probs = qnode()
        out_freq[idx] = qnode() * norm[idx]

    return out_freq

In [19]:
def devectorization(out_freq):
    S = out_freq.shape[0]
    nrow = int(np.sqrt(out_freq.shape[1])) # rows per patch
    ncol = nrow

    decoded_patches = np.reshape(out_freq,\
                      (out_freq.shape[0], nrow, ncol)) # (S, nrow, ncol)

    im_h, im_w = nrow*int(np.sqrt(S)), ncol*int(np.sqrt(S)) # final shape

    # initialization
    decoded_img = np.zeros((im_w, im_h))

    idx = 0
    for row in np.arange(im_h - nrow + 1, step=nrow):
        for col in np.arange(im_w - ncol + 1, step=ncol):
            decoded_img[row:row+nrow, col:col+ncol] = decoded_patches[idx]
            idx += 1

    return decoded_img

In [20]:
def qjpeg_feature_map_quantum(img_28x28):
    """
    True QJPEG-inspired feature map:
    - probabilities sum to 1
    - amplitudes = sqrt(probabilities)
    - output dimension = 64 (6 qubits)
    """

    img = img_28x28.astype(float)
    img = img / img.sum()              # probabilities
    amps = np.sqrt(img.flatten())      # amplitudes

    # reduce to 64 amplitudes (simple truncation for now)
    amps = amps[:64]

    # safety
    if np.linalg.norm(amps) == 0:
        amps[0] = 1.0
    else:
        amps /= np.linalg.norm(amps)

    return amps


### Step 4: Inference without retraining

In [21]:
### Step 4: Train expressive circuit and inference

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

C = 1.0
layers_list = [2, 4, 6, 8, 10, 12]

# --- Device setup ---
dev = qml.device(device, wires=n_qubits, shots=None)

# --- Define expressive circuit QNode ---
@qml.qnode(dev, interface="autograd")
def qnode_expressive(x, theta):
    # Amplitude embedding
    qml.AmplitudeEmbedding(x, wires=range(n_qubits), normalize=True)

    n_layers = theta.shape[0]
    for l in range(n_layers):
        for i in range(n_qubits):
            qml.RX(theta[l, i, 0], wires=i)
            qml.RY(theta[l, i, 1], wires=i)
            qml.RZ(theta[l, i, 2], wires=i)
        # Ring entanglement
        for i in range(n_qubits - 1):
            qml.CNOT(wires=[i, i + 1])
        qml.CNOT(wires=[n_qubits - 1, 0])
    return qml.state()  # return full quantum state

def qjpeg_to_32(img_28x28):
  """Return 32-dimensional QJPEG vector from 28x28 image."""
  amps = qjpeg_feature_map_quantum(img_28x28) # get 64 amplitudes
  amps_32 = amps[:32] # truncate to 32
  norm = np.linalg.norm(amps_32)
  if norm == 0:
    amps_32[0] = 1.0
    norm = 1.0
  return amps_32 / norm

# --- Initialize trainable parameters ---
n_layers_max = max(layers_list)
theta_trainable = 0.01 * pnp.random.randn(n_layers_max, n_qubits, 3, requires_grad=True)

# --- Kernel alignment loss function ---
def loss_fn(theta):
    return kernel_alignment_loss_batch(theta, X_train_qek, y_train, batch_size=24)

# --- Optimizer ---
opt = qml.AdamOptimizer(stepsize=0.01)
n_steps = 50

print("Training expressive circuit theta with kernel alignment...")
for step in range(n_steps):
    theta_trainable, loss = opt.step_and_cost(loss_fn, theta_trainable)
    if step % 5 == 0 or step == n_steps-1:
        print(f"Step {step}: loss = {loss:.4f}")

print("Training done.\n")

# --- Helper: combine QEK + QJPEG vectors ---
def combine_qek_qjpeg(x_qek, x_qjpeg):
    v = np.concatenate([x_qek[:32], x_qjpeg])
    norm = np.linalg.norm(v)
    if norm == 0:
        return v
    return v / norm

# --- Prepare QJPEG 32-dim vectors ---
X_train_qjpeg = np.array([qjpeg_to_32(img.reshape(28,28)) for img in X_train_img])
X_test_qjpeg  = np.array([qjpeg_to_32(img.reshape(28,28)) for img in X_test_img])

# --- Prepare combined vectors ---
X_train_combined = np.array([combine_qek_qjpeg(a, b) for a,b in zip(X_train_qek, X_train_qjpeg)])
X_test_combined  = np.array([combine_qek_qjpeg(a, b) for a,b in zip(X_test_qek, X_test_qjpeg)])

# --- Kernel computation ---
def compute_kernel(states_a, states_b=None):
    if states_b is None:
        states_b = states_a
    K = np.zeros((len(states_a), len(states_b)))
    for i, a in enumerate(states_a):
        for j, b in enumerate(states_b):
            K[i,j] = np.abs(np.vdot(a, b))**2
    return K

# --- Evaluate kernel with precomputed SVM ---
def evaluate_kernel(qnode, theta, X_train, X_test, y_train, y_test, C=1.0):
    states_train = np.array([qnode(x, theta) for x in X_train])
    states_test  = np.array([qnode(x, theta) for x in X_test])

    K_train = compute_kernel(states_train)
    max_val = np.max(K_train)
    if max_val == 0: max_val = 1.0
    K_train /= max_val

    K_test = compute_kernel(states_test, states_train)
    K_test /= max_val

    clf = SVC(kernel="precomputed", C=C)
    clf.fit(K_train, y_train)
    y_pred = clf.predict(K_test)
    return accuracy_score(y_test, y_pred)

# --- Evaluate different layer depths ---
results = []
for layers in layers_list:
    theta_used = theta_trainable[:layers]  # use first L layers

    print("\n" + "="*40)
    print(f"Evaluating expressive circuit with {layers} layers")
    print("="*40)

    # QEK only
    acc_qek = evaluate_kernel(
        qnode_expressive, theta_used,
        X_train_qek, X_test_qek,
        y_train, y_test, C=C
    )
    print(f"QEK accuracy:         {acc_qek:.4f}")

    # QEK + QJPEG combined
    acc_combined = evaluate_kernel(
        qnode_expressive, theta_used,
        X_train_combined, X_test_combined,
        y_train, y_test, C=C
    )
    print(f"QEK + QJPEG accuracy: {acc_combined:.4f}")

    results.append((layers, acc_qek, acc_combined))

print("\nAll results:", results)


Training expressive circuit theta with kernel alignment...
Step 0: loss = 36.7231
Step 5: loss = 34.5823
Step 10: loss = 33.4163
Step 15: loss = 23.3320
Step 20: loss = 32.6707
Step 25: loss = 34.7312
Step 30: loss = 21.1527
Step 35: loss = 36.1832
Step 40: loss = 36.0441
Step 45: loss = 26.7881
Step 49: loss = 30.0226
Training done.


Evaluating expressive circuit with 2 layers
QEK accuracy:         0.9500
QEK + QJPEG accuracy: 0.9500

Evaluating expressive circuit with 4 layers
QEK accuracy:         0.9500
QEK + QJPEG accuracy: 0.9500

Evaluating expressive circuit with 6 layers
QEK accuracy:         0.9500
QEK + QJPEG accuracy: 0.9500

Evaluating expressive circuit with 8 layers
QEK accuracy:         0.9500
QEK + QJPEG accuracy: 0.9500

Evaluating expressive circuit with 10 layers
QEK accuracy:         0.9500
QEK + QJPEG accuracy: 0.9500

Evaluating expressive circuit with 12 layers
QEK accuracy:         0.9500
QEK + QJPEG accuracy: 0.9500

All results: [(2, 0.95, 0.95), (4, 0.95, 0

In [22]:
C = 1.0
layers_list = [2, 4, 6, 8, 10, 12]

# --- Devices and QNodes ---
dev = qml.device(device, wires=n_qubits, shots=None)

# Expressive circuit QNode
qnode_expressive = qml.QNode(feature_map, dev)

n_layers_max = max(layers_list)

In [23]:
def combine_qek_qjpeg(x_qek, x_qjpeg):
    """Combine first 32 QEK amplitudes + full QJPEG vector, then normalize."""
    v = np.concatenate([x_qek[:32], x_qjpeg])
    norm = np.linalg.norm(v)
    if norm == 0:
        return v
    return v / norm

def compute_kernel(states_a, states_b=None):
    """Compute quantum kernel matrix from states."""
    if states_b is None:
        states_b = states_a
    K = np.zeros((len(states_a), len(states_b)))
    for i, a in enumerate(states_a):
        for j, b in enumerate(states_b):
            K[i,j] = np.abs(np.vdot(a, b))**2
    return K

def evaluate_kernel(qnode, theta, X_train, X_test, y_train, y_test, C=1.0):
    """Compute kernel, train SVM with precomputed kernel, and return test accuracy."""
    # Compute states
    states_train = np.array([qnode(x, theta) for x in X_train])
    states_test  = np.array([qnode(x, theta) for x in X_test])

    # Kernel computation with safe normalization
    K_train = compute_kernel(states_train)
    max_val = np.max(K_train)
    if max_val == 0:
        max_val = 1.0
    K_train /= max_val

    K_test = compute_kernel(states_test, states_train)
    K_test /= max_val

    # Train SVM
    clf = SVC(kernel="precomputed", C=C)
    clf.fit(K_train, y_train)

    # Predict and return accuracy
    y_pred = clf.predict(K_test)
    return accuracy_score(y_test, y_pred)

def qjpeg_to_32(img_28x28):
    """Return 32-dimensional QJPEG vector from 28x28 image."""
    amps = qjpeg_feature_map_quantum(img_28x28)  # get 64 amplitudes
    amps_32 = amps[:32]  # truncate to 32
    norm = np.linalg.norm(amps_32)
    if norm == 0:
        amps_32[0] = 1.0
        norm = 1.0
    return amps_32 / norm

In [24]:
# --- Prepare QJPEG 32-dimensional vectors ---
X_train_qjpeg = np.array([qjpeg_to_32(img.reshape(28,28)) for img in X_train_img])
X_test_qjpeg  = np.array([qjpeg_to_32(img.reshape(28,28)) for img in X_test_img])

# --- Prepare combined QEK + QJPEG vectors ---
X_train_combined = np.array([
    combine_qek_qjpeg(a, b) for a, b in zip(X_train_qek, X_train_qjpeg)
])
X_test_combined = np.array([
    combine_qek_qjpeg(a, b) for a, b in zip(X_test_qek, X_test_qjpeg)
])

# --- Random initial theta for expressive circuit ---
theta_base = np.random.uniform(
    0, 2*np.pi, size=(max(layers_list), n_qubits, 3)
)

# --- Evaluate circuits ---
results = []


In [25]:
for layers_count in layers_list:
    print("\n" + "="*40)
    print(f"Evaluating expressive circuit with {layers_count} layers")
    print("="*40)

    # Define a QNode that returns the state vector
    @qml.qnode(dev)
    def expressive_state_qnode(x, current_theta):
        feature_map(x, current_theta)
        return qml.state()

    # Use theta_base for different numbers of layers, slicing it
    current_theta_params = theta_base[:layers_count]

    # QEK only
    acc_qek = evaluate_kernel(
        expressive_state_qnode,
        current_theta_params,
        X_train_qek,
        X_test_qek,
        y_train,
        y_test,
        C=C
    )
    print(f"QEK accuracy:         {acc_qek:.4f}")

    # QEK + QJPEG combined
    acc_combined = evaluate_kernel(
        expressive_state_qnode,
        current_theta_params,
        X_train_combined,
        X_test_combined,
        y_train,
        y_test,
        C=C
    )
    print(f"QEK + QJPEG accuracy: {acc_combined:.4f}")

    results.append((layers_count, acc_qek, acc_combined))


Evaluating expressive circuit with 2 layers
QEK accuracy:         0.9500
QEK + QJPEG accuracy: 0.9500

Evaluating expressive circuit with 4 layers
QEK accuracy:         0.9500
QEK + QJPEG accuracy: 0.9500

Evaluating expressive circuit with 6 layers
QEK accuracy:         0.9500
QEK + QJPEG accuracy: 0.9500

Evaluating expressive circuit with 8 layers
QEK accuracy:         0.9500
QEK + QJPEG accuracy: 0.9500

Evaluating expressive circuit with 10 layers
QEK accuracy:         0.9500
QEK + QJPEG accuracy: 0.9500

Evaluating expressive circuit with 12 layers
QEK accuracy:         0.9500
QEK + QJPEG accuracy: 0.9500
